In [1]:
# data source: https://www.kaggle.com/alxmamaev/flowers-recognition

%reset -f

from keras.models import Sequential # to initialize CNN as a sequence of layers
from keras.layers import Convolution2D # for convolutional operations
from keras.layers import MaxPooling2D # for pooling operations
from keras.layers import Flatten # to flatten stacked feature maps into input layer
from keras.layers import Dense # to build fully-connected layers in a traditional neural network
from keras.layers import Dropout # to build a dropout layer(s); helps prevent overfitting

import numpy as np
np.random.seed(123)

C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
##### part 1: building the CNN

In [3]:
### initialize the CNN
classifier = Sequential()

In [4]:
### step 1: add a convolution layer to the CNN
classifier.add(Convolution2D(filters=32, # 32 filters/kernels
                             kernel_size=(3,3), # each kernel has a 3x3 receptive field
                             padding='same', # spatial dimensions of input image and feature map are the same
                             input_shape=(64,64,3), # each input image has width=height=64, and 3 color channels
                             strides=(1,1), # stride 1 pixel at a time, along the width and height
                             activation='relu')) # apply ReLU activation function to the conv layer element-wise

In [5]:
### step 2: add a pooling layer to the CNN
classifier.add(MaxPooling2D(pool_size=(2,2), # pooling kernel has a 2x2 receptive field
                            strides=(2,2))) # stride 2 pixels at a time, along the width and height

In [6]:
### step 3: repeat step 1 and 2 for a deep CNN
classifier.add(Convolution2D(filters=32, # 32 filters
                             kernel_size=(3,3), # each kernel has a 3x3 receptive field
                             padding='same', # spatial dimensions of input image and feature map are the same
                             strides=(1,1), # stride 1 pixel at a time, along the width and height
                             activation='relu')) # apply ReLU activation function to the conv layer element-wise

classifier.add(MaxPooling2D(pool_size=(2,2), # pooling kernel has a 2x2 receptive field
                            strides=(2,2))) # stride 2 pixels at a time, along the width and height

In [7]:
### step 4: flatten the pooled feature maps into a vector of neurons (to be used as an input layer in a classic NN)
classifier.add(Flatten())

In [8]:
### step 5: fully connect the input layer to a hidden layer and output layer (classic NN)
classifier.add(Dropout(0.5)) # dropout layer (with 50% probability of shutting down any given neuron) prevents overfitting

classifier.add(Dense(units=128, # 128 neurons in the hidden layer
                     activation='relu'))

classifier.add(Dropout(0.5)) # dropout layer (with 50% probability of shutting down any given neuron) prevents overfitting

classifier.add(Dense(units=5, # if classification problem was binary, then 1 unit
                     activation='softmax')) # if classification problem was binary, we'd use sigmoid

In [9]:
### compile the CNN
classifier.compile(optimizer='adam', # stochastic gradient descent algorithm (to optimize the weights)
                   loss='categorical_crossentropy',
                   metrics=['accuracy']) # alternative: crossentropy

In [10]:
##### part 2: fitting the CNN to the training set

In [11]:
### prevent overfitting by augmenting the number of images with transformed (e.g. zoomed, sheared) versions of those images
from keras.preprocessing.image import ImageDataGenerator

import os
os.chdir('D:\data')

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size=(64,64), # all images will be resized to 64x64 pixels
                                                 batch_size=32, # weights are updated every time 32 images have been fed into the CNN
                                                 class_mode='categorical') # multiclass classification

test_set = test_datagen.flow_from_directory('test_set',
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='categorical')

classifier.fit_generator(training_set,
                         steps_per_epoch=4073, # number of images in the training set
                         epochs=3, # number of times you want to feed all the training images into the CNN
                         validation_data=test_set,
                         validation_steps=250) # number of images in the test set

Found 4073 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Epoch 1/3
4073/4073 [==============================] - 3959s 972ms/step - loss: 0.7397 - acc: 0.7124 - val_loss: 0.7446 - val_acc: 0.7560
Epoch 2/3
4073/4073 [==============================] - 3438s 844ms/step - loss: 0.4400 - acc: 0.8345 - val_loss: 0.8155 - val_acc: 0.7560
Epoch 3/3
4073/4073 [==============================] - 3694s 907ms/step - loss: 0.3372 - acc: 0.8728 - val_loss: 0.9146 - val_acc: 0.7520
